In [ ]:
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import numpy as np
import os, sys
import warnings
warnings.filterwarnings('ignore')

from cloudbanddetection.tracking import plot_tracking_on_map
from cloudbanddetection.io_utilities import load_ymlfile, load_data_from_saved_var_files
from cloudbanddetection.time_utilities import create_list_of_dates, add_startend_datetime2config

#### Load configuration file and make adjustements for this test case

In [ ]:
config_file = "../config/config_analysis.yml"
config = load_ymlfile(config_file, isconfigfile=True)
# Make sure that the period for the cloud bands cover the same period as the example data
config["startdate"] = "20210219.00"
config["enddate"] = "20210228.00"
config["saved_dirpath"] = "../data/"

print("Example data exists:", os.path.isfile(config['saved_dirpath']+"/list_of_cloud_bands"+config['startdate']+"-"+config['enddate']+"-"+config['domain']+".pickle")) 


#### Load cloud bands from the South Pacific for the dates specified in the analyses config file

In [ ]:
add_startend_datetime2config(config)
listofdates = create_list_of_dates(config)
print(str(len(listofdates))+" dates created from "+str(listofdates[0])+" to "+str(listofdates[-1]))

In [ ]:
# Load one year of cloud bands
print(config)
list_of_cloud_bands = load_data_from_saved_var_files(config, varname="list_of_cloud_bands")
print(len(list_of_cloud_bands),"days of cloud bands")

#### Import longitudes and latitudes for the map

In [ ]:
lats_globe = np.load(dir_package + "data/lats_globe0.5_ERA5.npy")
lons_globe = np.load(dir_package + "data/lons_globe0.5_ERA5.npy")
# Get longitudes and latitudes of South Pacific domain
lon_ids = [id for id, el in enumerate(lons_globe) if el <= config["lon_east"] if el >= config["lon_west"]]
lonsSP = lons_globe[lon_ids]
lat_ids = [id for id, el in enumerate(lats_globe) if el <= config["lat_north"] if el >= config["lat_south"]]
latsSP = lats_globe[lat_ids]
print(f"Longitudes, min: {lonsSP.min()}, max: {lonsSP.max()}")
print(f"Latitudes, min: {latsSP.min()}, max: {latsSP.max()}")

#### Create the series of map with connection lines between the centroid lcoation of each cloud bands

In [ ]:
plot_tracking_on_map(
    list_of_cloud_bands=list_of_cloud_bands,
    lons=lonsSP,
    lats=latsSP,
    listofdates=listofdates,
    config=config,
    show=True,
    save=False,
)